In [2]:
import time
import numpy as np
import pandas as pd
import argparse
from copy import deepcopy
from scipy import interpolate
from sklearn.metrics import normalized_mutual_info_score
from scipy.stats import pearsonr
import scipy.sparse
import sys
import pickle
import re
from pyitlib import discrete_random_variable as drv

In [68]:
# # calculate Mutual Information of gene expression 
# def mutualMatrix(data, thresholdfold=3, bin=100):

#     total = np.zeros((data.shape[1], data.shape[1]))
    
#     for i in np.arange(data.shape[1]):
#         for j in np.arange(data.shape[1]):
#             total[i][j] = calc_MI(data[:,i], data[:,j], bin)
    
#     total = np.asarray(total)
#     threshold = np.mean(total)+thresholdfold*np.std(total)

#     for i in np.arange(data.shape[1]):
#         for j in np.arange(data.shape[1]):
#             if total[i][j] >= threshold:
#                 total[i][j] = 1.0
    
#     return total


# def calc_MI(x, y, bins):
#     mi = normalized_mutual_info_score(x, y, average_method='arithmetic')
#     return mi


def MI(X,Y):

    Cx = np.cov(X)
    Cy = np.cov(Y)
    
    Cxy = np.cov(X,Y)
    
    detCx = Cx
    detCy = Cy
    detCxy = np.linalg.det(Cxy)
    
    mul = (detCx*detCy)/detCxy
    
    logval = np.log(mul)
    
    total = 1/2 * logval
    
    return total



def CMI(X,Y,Z):

    XZ = np.array(X,Z)
    print(XZ.shape)
    YZ = np.array([Y,Z])
    XYZ = np.array([X,Y,Z])
    
    Cxz = np.cov(XZ)
    Cyz = np.cov(YZ)
    Cz = np.cov(Z)
    Cxyz = np.cov(XYZ)

    if Cxz.shape[0] > 1 or Cxz.shape[1] > 1:
        detCxz = np.linalg.det(Cxz)
    else:
        detCxz = Cxz
    if Cyz.shape[0] > 1 or Cyz.shape[1] > 1:
        detCyz = np.linalg.det(Cyz)
    else:
        detCyz = Cyz
#     if Cz.shape[0] > 1 or Cz.shape[1] > 1:
#         detCz = np.linalg.det(Cz)
#     else:
    detCz = Cz
    
    if Cxyz.shape[0] > 1 or Cxyz.shape[1] > 1:
        detCxyz = np.linalg.det(Cxyz)
    else:
        detCxyz = Cxyz
  

    
    mul = (detCxz*detCyz)/(detCz*detCxyz)
    
    logval = np.log(mul)
    
    total = 1/2 * logval
    
    return total


In [4]:
data = pd.read_csv('training data/Network 4 - S. cerevisiae/net4_expression_data.tsv', sep='\t')
data

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,...,G5941,G5942,G5943,G5944,G5945,G5946,G5947,G5948,G5949,G5950
0,8.3066,9.7335,10.345,9.1199,10.3480,9.6440,10.334,11.1420,8.5592,6.9191,...,10.7100,11.6830,7.0281,4.7497,6.7293,14.497,7.4510,11.001,7.7354,14.398
1,8.2621,9.8441,10.065,9.2828,10.3880,9.6187,10.453,11.1310,8.4064,7.1793,...,10.4120,11.6710,6.9203,4.8077,6.5953,14.449,7.7653,10.986,8.0687,14.338
2,8.2446,9.5841,10.150,10.5620,10.3130,9.9768,10.746,11.9670,9.6170,7.4095,...,10.4220,11.5730,7.1626,6.0187,7.3315,14.432,7.8711,10.372,8.1065,14.211
3,9.1807,9.8570,10.189,9.7156,10.5940,9.3862,10.692,11.0360,8.8417,7.7038,...,10.4540,11.5700,6.9334,5.9688,7.5520,14.207,7.9300,11.465,8.0246,14.314
4,9.1068,9.7898,10.150,9.3488,10.4410,9.6223,10.704,11.3960,8.7770,7.1017,...,10.1410,11.8350,7.4359,4.9402,7.1743,14.366,7.9784,11.440,7.7537,14.293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,9.7459,9.0307,11.545,6.8419,10.8110,9.4231,11.423,8.5443,8.6247,10.2170,...,10.2190,9.9715,8.2938,8.5158,10.3790,12.227,6.2391,12.055,8.3666,14.423
532,10.0830,8.7802,10.919,6.9747,10.0940,9.3423,12.018,9.2095,7.5045,9.5118,...,10.0880,10.2470,6.2689,8.3748,11.0390,12.839,6.5553,11.807,9.0364,14.320
533,10.2510,8.8318,10.729,7.5930,9.7559,9.2102,12.089,9.3490,7.4464,9.6238,...,8.7006,10.3630,6.1645,7.9322,11.3510,13.280,6.8444,11.479,9.2033,14.366
534,10.4020,8.1423,10.410,7.9926,9.1889,8.9210,12.008,9.2920,8.4852,9.4035,...,10.2260,10.4600,3.8795,8.0558,10.9830,12.912,6.5980,11.644,9.0416,14.308


In [5]:
benchmark_network = pd.read_csv('test data/DREAM5_NetworkInference_GoldStandard_Network4 - S. cerevisiae.tsv', sep='\t')
benchmark_network

,G15,G1,1
0,G98,G1,1
1,G109,G1,1
2,G22,G9,1
3,G211,G11,1
4,G41,G12,1
...,...,...,...
227196,G316,G5950,0
227197,G319,G5950,0
227198,G324,G5950,0
227199,G329,G5950,0


In [6]:
string = ''

for i in range(21,333):
    string = 'G'+ str(i)
    data.drop(string, axis=1, inplace = True)

data

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,...,G5941,G5942,G5943,G5944,G5945,G5946,G5947,G5948,G5949,G5950
0,8.3066,9.7335,10.345,9.1199,10.3480,9.6440,10.334,11.1420,8.5592,6.9191,...,10.7100,11.6830,7.0281,4.7497,6.7293,14.497,7.4510,11.001,7.7354,14.398
1,8.2621,9.8441,10.065,9.2828,10.3880,9.6187,10.453,11.1310,8.4064,7.1793,...,10.4120,11.6710,6.9203,4.8077,6.5953,14.449,7.7653,10.986,8.0687,14.338
2,8.2446,9.5841,10.150,10.5620,10.3130,9.9768,10.746,11.9670,9.6170,7.4095,...,10.4220,11.5730,7.1626,6.0187,7.3315,14.432,7.8711,10.372,8.1065,14.211
3,9.1807,9.8570,10.189,9.7156,10.5940,9.3862,10.692,11.0360,8.8417,7.7038,...,10.4540,11.5700,6.9334,5.9688,7.5520,14.207,7.9300,11.465,8.0246,14.314
4,9.1068,9.7898,10.150,9.3488,10.4410,9.6223,10.704,11.3960,8.7770,7.1017,...,10.1410,11.8350,7.4359,4.9402,7.1743,14.366,7.9784,11.440,7.7537,14.293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,9.7459,9.0307,11.545,6.8419,10.8110,9.4231,11.423,8.5443,8.6247,10.2170,...,10.2190,9.9715,8.2938,8.5158,10.3790,12.227,6.2391,12.055,8.3666,14.423
532,10.0830,8.7802,10.919,6.9747,10.0940,9.3423,12.018,9.2095,7.5045,9.5118,...,10.0880,10.2470,6.2689,8.3748,11.0390,12.839,6.5553,11.807,9.0364,14.320
533,10.2510,8.8318,10.729,7.5930,9.7559,9.2102,12.089,9.3490,7.4464,9.6238,...,8.7006,10.3630,6.1645,7.9322,11.3510,13.280,6.8444,11.479,9.2033,14.366
534,10.4020,8.1423,10.410,7.9926,9.1889,8.9210,12.008,9.2920,8.4852,9.4035,...,10.2260,10.4600,3.8795,8.0558,10.9830,12.912,6.5980,11.644,9.0416,14.308


In [7]:
highly_correlated = pd.DataFrame()

for j in range(1,21): 
    newdf = pd.DataFrame(data[data.columns[j:]].apply(lambda x: x.corr(data['G'+str(j)])))
    newdf[0] = newdf[0].abs()
    newdf = newdf.sort_values(by=0, ascending=False)
    newdf['gene'] = newdf.index
    highly_correlated = highly_correlated.append(newdf.head(50), ignore_index=True)
    
highly_correlated

,0,gene
0,0.655953,G4477
1,0.645280,G3029
2,0.643287,G3744
3,0.632333,G5329
4,0.622695,G3887
...,...,...
995,0.633610,G3748
996,0.632048,G3378
997,0.631490,G2944
998,0.631453,G2032


In [8]:
highly_correlated = highly_correlated.sort_values(by=0, ascending=False)

highly_correlated_filtered = highly_correlated.head(200)

highly_correlated_filtered = highly_correlated_filtered.drop_duplicates(subset=['gene'])

highly_correlated_filtered

,0,gene
100,0.932999,G1188
101,0.926053,G4099
102,0.912606,G4814
103,0.910448,G5532
104,0.909900,G2354
...,...,...
158,0.681689,G1230
825,0.681465,G5773
463,0.681254,G2556
865,0.680385,G2420


In [9]:
# https://stackoverflow.com/questions/2669059/how-to-sort-alpha-numeric-set-in-python

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [10]:
genes_to_retain = highly_correlated_filtered['gene'].to_list()

for i in range(1,21):
    genes_to_retain.append('G'+str(i))
    
genes_to_retain = list(set(genes_to_retain))

genes_to_retain = sorted_nicely(genes_to_retain)

genes_to_retain


['G1',
 'G2',
 'G3',
 'G4',
 'G5',
 'G6',
 'G7',
 'G8',
 'G9',
 'G10',
 'G11',
 'G12',
 'G13',
 'G14',
 'G15',
 'G16',
 'G17',
 'G18',
 'G19',
 'G20',
 'G384',
 'G406',
 'G420',
 'G470',
 'G472',
 'G600',
 'G625',
 'G662',
 'G751',
 'G800',
 'G812',
 'G835',
 'G854',
 'G913',
 'G1041',
 'G1058',
 'G1093',
 'G1131',
 'G1132',
 'G1136',
 'G1187',
 'G1188',
 'G1230',
 'G1239',
 'G1255',
 'G1287',
 'G1302',
 'G1308',
 'G1320',
 'G1346',
 'G1350',
 'G1352',
 'G1355',
 'G1362',
 'G1440',
 'G1521',
 'G1558',
 'G1562',
 'G1576',
 'G1598',
 'G1607',
 'G1630',
 'G1672',
 'G1674',
 'G1722',
 'G1725',
 'G1728',
 'G1789',
 'G1803',
 'G1811',
 'G1839',
 'G1856',
 'G1918',
 'G1942',
 'G1966',
 'G1969',
 'G1971',
 'G2047',
 'G2053',
 'G2059',
 'G2082',
 'G2124',
 'G2195',
 'G2281',
 'G2339',
 'G2354',
 'G2386',
 'G2415',
 'G2420',
 'G2421',
 'G2449',
 'G2460',
 'G2469',
 'G2475',
 'G2480',
 'G2491',
 'G2496',
 'G2525',
 'G2556',
 'G2564',
 'G2569',
 'G2584',
 'G2602',
 'G2638',
 'G2646',
 'G2671',
 'G

In [14]:
preprocessed_data = pd.DataFrame()

preprocessed_data = data[genes_to_retain]
preprocessed_data

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,...,G5722,G5743,G5771,G5773,G5785,G5808,G5810,G5824,G5845,G5858
0,8.3066,9.7335,10.345,9.1199,10.3480,9.6440,10.334,11.1420,8.5592,6.9191,...,11.509,10.5320,10.2120,11.8400,10.1800,7.7837,11.4920,10.3740,7.1004,11.281
1,8.2621,9.8441,10.065,9.2828,10.3880,9.6187,10.453,11.1310,8.4064,7.1793,...,11.494,10.6740,10.1090,11.8660,10.4440,7.5726,11.3950,10.3740,7.0592,11.295
2,8.2446,9.5841,10.150,10.5620,10.3130,9.9768,10.746,11.9670,9.6170,7.4095,...,11.642,10.3850,9.8995,11.0010,9.8615,7.7017,11.2940,10.2650,6.8129,11.461
3,9.1807,9.8570,10.189,9.7156,10.5940,9.3862,10.692,11.0360,8.8417,7.7038,...,11.154,10.7710,10.0320,11.5630,10.3980,7.9657,11.3880,10.2610,7.3716,11.743
4,9.1068,9.7898,10.150,9.3488,10.4410,9.6223,10.704,11.3960,8.7770,7.1017,...,11.376,10.7050,10.2150,11.8270,10.2600,7.4790,11.4330,10.2240,6.7096,11.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,9.7459,9.0307,11.545,6.8419,10.8110,9.4231,11.423,8.5443,8.6247,10.2170,...,11.380,9.3354,7.8509,8.4495,8.4629,11.3420,9.6999,6.4959,8.1935,12.652
532,10.0830,8.7802,10.919,6.9747,10.0940,9.3423,12.018,9.2095,7.5045,9.5118,...,11.143,9.2828,8.3525,9.0859,8.1528,10.8880,8.0726,5.8112,8.1916,12.867
533,10.2510,8.8318,10.729,7.5930,9.7559,9.2102,12.089,9.3490,7.4464,9.6238,...,11.101,9.1867,8.0961,9.3603,8.2647,10.6790,8.0021,6.4273,8.1509,13.146
534,10.4020,8.1423,10.410,7.9926,9.1889,8.9210,12.008,9.2920,8.4852,9.4035,...,10.637,9.0761,7.7991,8.5783,7.8706,10.5480,7.9409,6.5834,6.3479,12.955


In [46]:
# m = mutualMatrix(preprocessed_data_np)
# m

preprocessed_data_np = preprocessed_data.to_numpy()

# mi = MI(preprocessed_data_np[:,1], preprocessed_data_np[:,1])
# print(mi)

miMat = np.zeros((preprocessed_data_np.shape[1],preprocessed_data_np.shape[1]))

for i in range(0,preprocessed_data_np.shape[1]):
    for j in range(0,preprocessed_data_np.shape[1]):
        miMat[i][j] = round(MI(preprocessed_data_np[:,i], preprocessed_data_np[:,j]),2)

        
miMat


/work/LAS/mash-lab/harshavk/conda/envs/scrna/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars


array([[inf, 1.000000, 1.250000, ..., 1.050000, 1.130000, 1.120000],
       [1.000000, inf, 1.020000, ..., 1.000000, 1.040000, 1.020000],
       [1.250000, 1.020000, inf, ..., 1.120000, 3.540000, 1.010000],
       ...,
       [1.050000, 1.000000, 1.120000, ..., inf, 1.200000, 1.070000],
       [1.130000, 1.040000, 3.540000, ..., 1.200000, inf, 1.000000],
       [1.120000, 1.020000, 1.010000, ..., 1.070000, 1.000000, inf]])

In [44]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

miMat

array([[inf, 0.002200, 0.111700, ..., 0.025700, 0.061100, 0.054500],
       [0.002200, inf, 0.010900, ..., 0.000100, 0.021700, 0.009600],
       [0.111700, 0.010900, inf, ..., 0.057800, 0.632300, 0.002900],
       ...,
       [0.025700, 0.000100, 0.057800, ..., inf, 0.089200, 0.034800],
       [0.061100, 0.021700, 0.632300, ..., 0.089200, inf, 0.000200],
       [0.054500, 0.009600, 0.002900, ..., 0.034800, 0.000200, inf]])

In [69]:
CMI(preprocessed_data_np[:,0], preprocessed_data_np[:,1], preprocessed_data_np[:,2])

TypeError: Cannot construct a dtype from an array